This notebook uses a deep learning image classification model, VGG16, to find similar images among a set of photos of household tools, cheese graters and foot files.

Below we import the libraries we'll be using.

In [ ]:
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np 

We'll download the model VGG16 that has already been trained on the dataset ImageNet.

In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=True)

The VGG16 model has 16 trainable layers, plus a few non-trainable layers.
Its input is a 224x224x3 tensor, and its output is a one-dimensional tensor of size 1000.
Use `model.summary()` to see a summary of its layers.

Next, we'll remove the last two layers of the model,
corresponding to the layers that use recognized features to generate classification.
After the removal, the output is a one-dimensional tensor of size 4096.

In [ ]:
model2 = Model(model.input, model.layers[-2].output)

We'll extract addtional images below.
The following code block extracts 20 image files to the current directory.

* cg01.jpeg through cg10.jpeg contain images of cheese graters.
* ff01.jpeg through ff10.jpeg contain images of foot files.

These images have size 1080x720, but we will downsize them to 224x224 below
to be compatible with the VGG16 model.

In [ ]:
import zipfile
with zipfile.ZipFile('imgs/tools.zip') as f:
    f.extractall()

We will then load and process the images so that we can input them into our model and use the predictions from the model to find pairs of images that are most similar to each other according to our model.

In [ ]:
# get images
from PIL import Image
from IPython.display import display

# dat will contain 20 one-dimensional numpy.ndarray, each of size 4096,
# corresponding to the 4096 features output by the truncated VGG16 model.
dat = []
# imgs will contain 20 PIL.Image.Image objects.
imgs = []
imgflist = ["cg01", "cg02", "cg03", "cg04", "cg05", "cg06", "cg07", "cg08", "cg09", "cg10",
           "ff01", "ff02", "ff03", "ff04", "ff05", "ff06", "ff07", "ff08", "ff09", "ff10"]
for imgf in imgflist:
    img = image.load_img(imgf+".jpeg", target_size=(224,224))
    imgs.append(img)
    img_arr = np.expand_dims(image.img_to_array(img), axis=0)
    x = preprocess_input(img_arr)
    preds = model2.predict(x)
    dat.append(preds[0])
for i in range(len(dat)):
  i1 = dat[i]
  bestmatch, bestsim = -1, 0
  for j in range(len(dat)):
    i2 = dat[j]
    sim = i1 @ i2
    if sim > bestsim and i != j: bestmatch, bestsim = j, sim
    print(j, sim)
  print(i, bestmatch, bestsim)
  display(imgs[i], imgs[bestmatch])

Below are the number of images we have downloaded

In [ ]:
len(dat)

We will also do this by dividing the data into training and testing data, and for each image in the training set, find the image in the test set that is most similar.

In [ ]:
train = [i for i in range(0,5)]+[i for i in range(10,15)]
test = [i for i in range(5,10)]+[i for i in range(15,20)]

for i in test:
  bestj = -1
  bestdist = 0
  for j in train:
    if i != j: 
#      dist = sum((dat[i] - dat[j])**2)
      dist = (dat[i] @ dat[j])**2 / ((dat[i] @ dat[i])*(dat[j] @ dat[j])) 
      if dist > bestdist: bestj, bestdist = j, dist
  print(f"test image {i} best matches train image {bestj}")
  display(imgs[i])
  display(imgs[bestj])
  print(" ")